# 환경 설정

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

# Retrieval tool

## retrieval tool 1
- using PDFRetrievalChain

In [ ]:
from rag.pdf import PDFRetrievalChain

# PDF 문서를 로드합니다.
pdf = PDFRetrievalChain(["data/example.pdf"]).create_chain()

# retriever와 chain을 생성합니다.
pdf_retriever = pdf.retriever
pdf_chain = pdf.chain

In [ ]:
from langchain_core.tools.retriever import create_retriever_tool
from langchain_core.prompts import PromptTemplate

# PDF 문서를 기반으로 검색 도구 생성
retriever_tool = create_retriever_tool(
    pdf_retriever,
    "pdf_retriever",
    "use this tool to search information from the PDF document.",
    document_prompt=PromptTemplate.from_template(
        "<document><context>{page_content}</context><metadata><source>{source}</source><page>{page}</page></metadata></document>"
    ),
)

# 생성된 검색 도구를 도구 리스트에 추가하여 에이전트에서 사용 가능하도록 설정
tools = [retriever_tool]


## Retrieval 2
- using multi modal pkl data
- 사전에 이미 pdf를 pkl로 변환함

In [3]:
import pickle
import os

def load_documents_from_pkl(filepath):
    """
    Pickle 파일에서 Langchain Document 리스트를 불러오는 함수

    Args:
        filepath: 원본 파일 경로 (예: path/to/filename.pdf)
    Returns:
        Langchain Document 객체 리스트
    """
    # 확장자 제거하고 절대 경로로 변환
    abs_path = os.path.abspath(filepath)
    base_path = os.path.splitext(abs_path)[0]
    pkl_path = f"{base_path}.pkl"

    with open(pkl_path, "rb") as f:
        documents = pickle.load(f)
    return documents

In [ ]:
import glob
from pathlib import Path

# extract_path 디렉토리에서 모든 .pkl 파일 찾기
extract_path = "data/"
pkl_files = glob.glob(str(Path(extract_path) / "**" / "*.pkl"), recursive=True)

if not pkl_files:
    print("❌ extract_path에서 .pkl 파일을 찾을 수 없습니다.")
else:
    # 모든 .pkl 파일에서 문서 로드
    all_documents = []
    for pkl_file in pkl_files:
        print(f"📄 {pkl_file} 파일 로드 중...")  # 한국어 코멘트
        documents = load_documents_from_pkl(pkl_file)
        all_documents.extend(documents)

    print(f"✅ 총 {len(all_documents)}개의 문서가 로드되었습니다.")

📄 /root/Project/aayn/teddynote-parser-api-client/example/parsing_outputs/84955c6c-9975-47cd-a4af-b9da78d548a5/84955c6c-9975-47cd-a4af-b9da78d548a5/84955c6c-9975-47cd-a4af-b9da78d548a5_RetirementPensionSubscriberTraining.pkl 파일 로드 중...
📄 /root/Project/aayn/teddynote-parser-api-client/example/parsing_outputs/a7116040-b8b7-4ea2-b096-98bc7a3ae049/a7116040-b8b7-4ea2-b096-98bc7a3ae049/a7116040-b8b7-4ea2-b096-98bc7a3ae049_WM_Pension_Investment_Report.pkl 파일 로드 중...
📄 /root/Project/aayn/teddynote-parser-api-client/example/parsing_outputs/aa347840-ec43-41a5-8439-94268bab19d4/aa347840-ec43-41a5-8439-94268bab19d4/aa347840-ec43-41a5-8439-94268bab19d4_Shinhan_Investment_Securities_Retirement_Pension.pkl 파일 로드 중...
✅ 총 142개의 문서가 로드되었습니다.


In [ ]:
# 문서 내용 확인
all_documents

[Document(metadata={'page': 0, 'source': '84955c6c-9975-47cd-a4af-b9da78d548a5_RetirementPensionSubscriberTraining.pdf'}, page_content='퇴직연금\n가입자 교육'),
 Document(metadata={'image': './images/figure/84955c6c-9975-47cd-a4af-b9da78d548a5_RetirementPensionSubscriberTraining_Page_0_Index_2.png)', 'entity': '<image>\n<title>\n경제 성장과 재정 관리의 시각화\n</title>\n<details>\n이미지는 경제 성장과 재정 관리를 상징적으로 표현하고 있으며, 사람들은 돈을 들고 있거나 쌓아놓고 있는 모습이 보입니다. 배경에는 도시 건물과 식물들이 있어 지속 가능한 발전을 암시합니다.\n</details>\n<entities>\n돈, 건물, 식물, 계산기\n</entities>\n<hypothetical_questions>\n- 현대 사회에서 재정 관리는 어떻게 변화하고 있을까?\n- 지속 가능한 경제 성장을 위해 개인이 할 수 있는 일은 무엇일까?\n</hypothetical_questions>\n</image>', 'page': 0, 'source': '84955c6c-9975-47cd-a4af-b9da78d548a5_RetirementPensionSubscriberTraining.pdf'}, page_content='![](./images/figure/84955c6c-9975-47cd-a4af-b9da78d548a5_RetirementPensionSubscriberTraining_Page_0_Index_2.png)\n\n<image>\n<title>\n경제 성장과 재정 관리의 시각화\n</title>\n<details>\n이미지는 경제 성장과 재정 관리를 상징적으로 표현하고 있으며, 사람들은 돈을 들고 있거나 쌓아

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools.retriever import create_retriever_tool

# VectorStore를 생성합니다.
vector = FAISS.from_documents(all_documents, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

tools = [retriever_tool]

# langgraph

In [8]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages


# 에이전트 상태를 정의하는 타입 딕셔너리, 메시지 시퀀스를 관리하고 추가 동작 정의
class AgentState(TypedDict):
    # add_messages reducer 함수를 사용하여 메시지 시퀀스를 관리
    messages: Annotated[Sequence[BaseMessage], add_messages]


## node and edge

In [ ]:
from typing import Literal
from langchain import hub
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import tools_condition

# 모델이름 가져오기
MODEL_NAME = "gpt-4o-mini"


# 데이터 모델 정의
class grade(BaseModel):
    """A binary score for relevance checks"""

    binary_score: str = Field(
        description="Response 'yes' if the document is relevant to the question or 'no' if it is not."
    )


def grade_documents(state) -> Literal["generate", "rewrite"]:
    # LLM 모델 초기화
    model = ChatOpenAI(temperature=0, model=MODEL_NAME, streaming=True)

    # 구조화된 출력을 위한 LLM 설정
    llm_with_tool = model.with_structured_output(grade)

    # 프롬프트 템플릿 정의
    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document 
        to a user question. \n 
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keyword(s) or semantic meaning related to the 
        user question, grade it as relevant. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document 
        is relevant to the question.""",
        input_variables=["context", "question"],
    )

    # llm + tool 바인딩 체인 생성
    # | 체이닝 연산자
    # 입력을 받아서 프롬프트를 생성(프롬프트 템플릿) -> llm에서 구조화된 결과로 출력(llm_with_tool)
    # 추후 invoke() 메서드로 호출 실행
    chain = prompt | llm_with_tool

    # 현재 상태에서 메시지 추출
    messages = state["messages"]

    # 가장 마지막 메시지 추출
    last_message = messages[-1]

    # 원래 질문 추출
    question = messages[0].content

    # 검색된 문서 추출
    retrieved_docs = last_message.content

    # 관련성 평가 실행
    scored_result = chain.invoke({"question": question, "context": retrieved_docs})

    # 관련성 여부 추출
    score = scored_result.binary_score

    # 관련성 여부에 따른 결정
    if score == "yes":
        print("==== [DECISION: DOCS RELEVANT] ====")
        return "generate"

    else:
        print("==== [DECISION: DOCS NOT RELEVANT] ====")
        print(score)
        return "rewrite"


def agent(state):
    # 현재 상태에서 메시지 추출
    messages = state["messages"]

    # LLM 모델 초기화
    model = ChatOpenAI(temperature=0, streaming=True, model=MODEL_NAME)

    # retriever tool 바인딩
    model = model.bind_tools(tools)

    # 에이전트 응답 생성
    response = model.invoke(messages)

    # 기존 리스트에 추가되므로 리스트 형태로 반환
    return {"messages": [response]}


def rewrite(state):
    print("==== [QUERY REWRITE] ====")
    # 현재 상태에서 메시지 추출
    messages = state["messages"]
    # 원래 질문 추출
    question = messages[0].content

    # 질문 개선을 위한 프롬프트 구성
    msg = [
        HumanMessage(
            content=f""" \n 
    Look at the input and try to reason about the underlying semantic intent / 
    meaning. \n 
    Here is the initial question:
    \n ------- \n
    {question} 
    \n ------- \n
    Formulate an improved question: """,
        )
    ]

    # LLM 모델로 질문 개선
    model = ChatOpenAI(temperature=0, model=MODEL_NAME, streaming=True)
    # Query-Transform 체인 실행
    response = model.invoke(msg)

    # 재작성된 질문 반환
    return {"messages": [response]}


def generate(state):
    # 현재 상태에서 메시지 추출
    messages = state["messages"]

    # 원래 질문 추출
    question = messages[0].content

    # 가장 마지막 메시지 추출
    docs = messages[-1].content

    # RAG 프롬프트 템플릿 가져오기
    prompt = hub.pull("teddynote/rag-prompt")

    # LLM 모델 초기화
    llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0, streaming=True)

    # RAG 체인 구성
    rag_chain = prompt | llm | StrOutputParser()

    # 답변 생성 실행
    response = rag_chain.invoke({"context": docs, "question": question})
    return {"messages": [response]}


## 그래프 생성

In [11]:
# LangGraph 라이브러리의 그래프 및 도구 노드 컴포넌트 임포트
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

# AgentState 기반 상태 그래프 워크플로우 초기화
workflow = StateGraph(AgentState)

# 워크플로우 내 순환 노드 정의 및 추가
workflow.add_node("agent", agent)  # 에이전트 노드
retrieve = ToolNode([retriever_tool])
workflow.add_node("retrieve", retrieve)  # 검색 노드
workflow.add_node("rewrite", rewrite)  # 질문 재작성 노드
workflow.add_node("generate", generate)  # 관련 문서 확인 후 응답 생성 노드

# 시작점에서 에이전트 노드로 연결
workflow.add_edge(START, "agent")

# 검색 여부 결정을 위한 조건부 엣지 추가
workflow.add_conditional_edges(
    "agent",
    # 에이전트 결정 평가
    tools_condition,
    {
        # 조건 출력을 그래프 노드에 매핑
        "tools": "retrieve",
        END: END,
    },
)

# 액션 노드 실행 후 처리될 엣지 정의
workflow.add_conditional_edges(
    "retrieve",
    # 문서 품질 평가
    grade_documents,
)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")

# 워크플로우 그래프 컴파일
graph = workflow.compile()


In [ ]:
from langgraph.graph.state import CompiledStateGraph
from langchain_core.runnables import RunnableConfig
from typing import Any, Dict, List, Callable

def stream_graph(
    graph: CompiledStateGraph,
    inputs: dict,
    config: RunnableConfig,
    node_names: List[str] = [],
    callback: Callable = None,
):
    """
    LangGraph의 실행 결과를 스트리밍하여 출력하는 함수입니다.

    Args:
        graph (CompiledStateGraph): 실행할 컴파일된 LangGraph 객체
        inputs (dict): 그래프에 전달할 입력값 딕셔너리
        config (RunnableConfig): 실행 설정
        node_names (List[str], optional): 출력할 노드 이름 목록. 기본값은 빈 리스트
        callback (Callable, optional): 각 청크 처리를 위한 콜백 함수. 기본값은 None
            콜백 함수는 {"node": str, "content": str} 형태의 딕셔너리를 인자로 받습니다.
    Returns:
        None: 함수는 스트리밍 결과를 출력만 하고 반환값은 없습니다.
    """
    prev_node = ""
    for chunk_msg, metadata in graph.stream(inputs, config, stream_mode="messages"):
        curr_node = metadata["langgraph_node"]

        # node_names가 비어있거나 현재 노드가 node_names에 있는 경우에만 처리
        if not node_names or curr_node in node_names:
            # 콜백 함수가 있는 경우 실행
            if callback:
                callback({"node": curr_node, "content": chunk_msg.content})
            # 콜백이 없는 경우 기본 출력
            else:
                # 노드가 변경된 경우에만 구분선 출력
                if curr_node != prev_node:
                    print("\n" + "=" * 50)
                    print(f"🔄 Node: \033[1;36m{curr_node}\033[0m 🔄")
                    print("- " * 25)
                print(chunk_msg.content, end="", flush=True)

            prev_node = curr_node

In [ ]:
from langchain_core.runnables import RunnableConfig

# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": "1"})

# 사용자의 에이전트 메모리 유형에 대한 질문을 포함하는 입력 데이터 구조 정의
inputs = {
    "messages": [
        ("user", "퇴직연금을 중간에 뽑을 수 있는 상황에 대해서 알려줘."),
    ]
}

# 그래프 실행
stream_graph(graph, inputs, config, ["agent", "rewrite", "generate"])



🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
==== [DECISION: DOCS RELEVANT] ====


/root/Project/aayn/.venv/lib/python3.10/site-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(



🔄 Node: generate 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
퇴직연금을 중간에 뽑을 수 있는 상황은 법정 사유와 요건을 갖춘 경우에 한하며, 적립금의 100% 한도 내에서 중도인출이 가능합니다. 중도인출은 DC 및 IRP 제도 가입자만 신청할 수 있습니다. 예를 들어, 본인 또는 부양가족의 대학등록금, 혼례비, 장례비 발생 시 중도인출이 가능합니다. 

**Source**
- 퇴직연금 중도인출 및 담보제공 (중도인출 · 담보대출 요건과 사유)